In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class Arm:
    def __init__(self, p):
        self.p = p
    
    def pull(self):
        return np.random.binomial(1, self.p)

In [ ]:
class MultiBandit:
    def __init__(self, probs=[0.1, 0.2, 0.7, 0.4]):
        self.__arms = [Arm(p) for p in probs]
        self.__regret = 0
        self.__maxp = max(probs)

    def num_arms(self):
        return len(self.__arms)

    def pull(self, arm_num):
        reward = self.__arms[arm_num].pull()
        self.__regret += self.__maxp - self.__arms[arm_num].p
        return reward
    
    def regret(self):
        return self.__regret

In [ ]:
class epsilongreedyalgorithm:
    def __init__(self, num_arms, horizon, epsilon):
        self.num_arms = num_arms
        self.horizon = horizon
        self.epsilon = epsilon
        self.timestep = 0
        self.arm_pulls = np.zeros(num_arms)
        self.arm_rewards = np.zeros(num_arms)
        self.regrets = np.zeros(horizon)
        self.best_arm = None

    def give_best_arm(self):
        return self.best_arm

    def select_arm(self):
        if np.random.rand() < self.epsilon:
           
            return np.random.choice(self.num_arms)
        else:
          
            return np.argmax(self.arm_rewards / (self.arm_pulls + 1e-5))

    def run_algorithm(self, bandit):
        for _ in range(self.horizon):
            arm_to_pull = self.select_arm()
            reward = bandit.pull(arm_to_pull)
            self.arm_pulls[arm_to_pull] += 1
            self.arm_rewards[arm_to_pull] += reward
            self.timestep += 1
            self.regrets[_] = bandit.regret()

        self.best_arm = np.argmax(self.arm_rewards / (self.arm_pulls + 1e-5))

    def plot(self, label):
        plt.plot(np.cumsum(self.regrets), label=label)

In [ ]:
class ucbalgorithm:
    def __init__(self, num_arms, horizon, c):
        self.num_arms = num_arms
        self.horizon = horizon
        self.c = c  # Exploration-exploitation tradeoff parameter
        self.timestep = 0
        self.arm_pulls = np.zeros(num_arms)
        self.arm_rewards = np.zeros(num_arms)
        self.regrets = np.zeros(horizon)
        self.best_arm = None

    def give_best_arm(self):
        return self.best_arm

    def select_arm(self):
       
        if self.timestep < self.num_arms:
            return self.timestep
        ucb_values = self.arm_rewards / (self.arm_pulls + 1e-5) + self.c * np.sqrt(np.log(self.timestep) / (self.arm_pulls + 1e-5))
        return np.argmax(ucb_values)

    def run_algorithm(self, bandit):
        for _ in range(self.horizon):
            arm_to_pull = self.select_arm()
            reward = bandit.pull(arm_to_pull)
            self.arm_pulls[arm_to_pull] += 1
            self.arm_rewards[arm_to_pull] += reward
            self.timestep += 1
            self.regrets[_] = bandit.regret()

        self.best_arm = np.argmax(self.arm_rewards / (self.arm_pulls + 1e-5))

    def plot(self, label):
        plt.plot(np.cumsum(self.regrets), label=label)

In [ ]:
class thompsonsamplingalgorithm:
    def __init__(self, num_arms, horizon):
        self.num_arms = num_arms
        self.horizon = horizon
        self.timestep = 0
        self.arm_successes = np.ones(num_arms)
        self.arm_failures = np.ones(num_arms)
        self.regrets = np.zeros(horizon)
        self.best_arm = None

    def give_best_arm(self):
        return self.best_arm

    def select_arm(self):
        samples = np.random.beta(self.arm_successes + 1, self.arm_failures + 1)
        return np.argmax(samples)

    def run_algorithm(self, bandit):
        for t in range(self.horizon):
            arm_to_pull = self.select_arm()
            reward = bandit.pull(arm_to_pull)
            if reward == 1:
                self.arm_successes[arm_to_pull] += 1
            else:
                self.arm_failures[arm_to_pull] += 1
            self.timestep += 1
            self.regrets[t] = bandit.regret()  
        self.best_arm = np.argmax(self.arm_successes + self.arm_failures)
    
    def plot(self, label):
        plt.plot(np.cumsum(self.regrets), label=label)

In [ ]:
def run_experiment(bandit, algorithms, horizon):
    for algorithm in algorithms:
        bandit = MultiBandit()
        algorithm.run_algorithm(bandit)
        total_regret = bandit.regret()
        best_arm = algorithm.give_best_arm()
        print(f"Total Regret after {horizon} timesteps: {total_regret} with assumed best arm {best_arm}")
        algorithm.plot(label=f'{algorithm.__class__.__name__}')

    plt.xlabel("Timestep")
    plt.ylabel("Total Regret")
    plt.title("Comparison of Algorithms")
    plt.legend()
    plt.show()

In [ ]:
bandit = MultiBandit()

In [ ]:
H = 1000

In [ ]:
epsilon_greedy = epsilongreedyalgorithm(num_arms=bandit.num_arms(), horizon=H, epsilon=0.2)
ucb_algorithm = ucbalgorithm(num_arms=bandit.num_arms(), horizon=H, c=0.18)
thompson_sampling = thompsonsamplingalgorithm(num_arms=bandit.num_arms(), horizon=H)

In [ ]:
algorithms_to_run = [epsilon_greedy, ucb_algorithm, thompson_sampling]

In [ ]:
run_experiment(bandit, algorithms_to_run, H)